# Surface water mapping

## Setup

In [1]:
from google.colab import drive
import ee
import geemap
import geopandas as gpd
import glob
import datetime
import pandas as pd

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/surface_water_mapping

/content/drive/MyDrive/surface_water_mapping


In [4]:
user_id = 'ee-alexvmt'
ee.Authenticate()
ee.Initialize(project=user_id)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [5]:
folder_name = 'surface_water_mapping'

In [6]:
# decide whether to export vectors to Drive
# True: if you haven't exported them yet
# False: if you only want to create the final shapefile
export = False

## Define date range of interest and region of interest

In [7]:
start_date = '2016-11-01'
end_date = '2017-04-30'

In [8]:
# Makgadikgadi Pan
roi_name = 'makgadikgadi_pan'

long_min = 24.51122
long_max = 25.41764
lat_min = -20.7972
lat_max = -20.40985

roi = ee.Geometry.Polygon([[
    [long_min, lat_max],
    [long_min, lat_min],
    [long_max, lat_min],
    [long_max, lat_max]
]])

## Load Sentinel-1 collection

In [9]:
col_filter = ee.Filter.And(
    ee.Filter.date(ee.Date(start_date), ee.Date(end_date)),
    ee.Filter.bounds(roi),
    ee.Filter.eq('instrumentMode', 'IW'),
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING'),
    ee.Filter.eq('resolution_meters', 10),
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
)

s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filter(col_filter)

In [10]:
s1

In [11]:
m = geemap.Map()
m.center_object(roi, 10)
m.add_layer(
    s1.first().clip(roi),
    {'bands': 'VV', 'min': -20, 'max': 0},
    'S1 image',
)
m

Map(center=[-20.60403337455113, 24.96442999999993], controls=(WidgetControl(options=['position', 'transparent_…

## Filter speckle noise

In [12]:
def filter_speckles(image):
  vv = image.select('VV')
  vv_smoothed = vv.focal_median(50, 'circle', 'meters').rename('VV_smoothed')
  return image.addBands(vv_smoothed)

In [13]:
s1_filtered = s1.map(filter_speckles)

In [14]:
s1_filtered

## Classify water

In [15]:
def classify_water(image):
  vv_smoothed = image.select('VV_smoothed')
  water = vv_smoothed.lt(-16.5616).rename('water')
  water = water.updateMask(water)
  return image.addBands(water)

In [16]:
s1_classified = s1_filtered.map(classify_water)

In [17]:
s1_classified

In [18]:
m = geemap.Map()
m.center_object(roi, 10)
m.add_layer(
    s1_filtered.first().clip(roi),
    {'bands': 'VV', 'min': -20, 'max': 0},
    'Filtered S1 image',
)
m.add_layer(
    s1_classified.first().clip(roi),
    {'bands': 'water', 'min': 0, 'max': 1, 'palette': ['#FFFFFF', '#0000FF']},
    'Water',
)
m

Map(center=[-20.60403337455113, 24.96442999999993], controls=(WidgetControl(options=['position', 'transparent_…

## Reduce classified images to vectors and start export

In [19]:
s1_classified_list = s1_classified.toList(s1_classified.size())
s1_classified_list_size = s1_classified_list.size().getInfo()

In [20]:
if export:
  for i in range(s1_classified_list_size):

    print('Processing image {}/{}...'.format(i+1, s1_classified_list_size))

    # get ith image from collection
    image = ee.Image(s1_classified_list.get(i))

    # generate file name
    date = image.date().format("YYYY-MM-dd").getInfo().replace('-', '')
    file_name = roi_name + '_' + date

    # reduce image to vectors
    vectors = image.select('water').reduceToVectors(
      reducer=ee.Reducer.countEvery(),
      geometry=roi,
      scale=10,
      geometryType='polygon',
      labelProperty='water',
      crs='EPSG:4326',
      maxPixels=1e13
    )

    # export vectors to shapefile
    task = ee.batch.Export.table.toDrive(
      collection=vectors,
      description=file_name,
      folder=folder_name,
      fileFormat='SHP'
    )
    task.start()

You can check the progress of your export here: https://code.earthengine.google.com/tasks

## Load and combine exported shape files
Make sure all shape files have been successfully exported before trying to combine them into one single shape file.

In [21]:
shape_files = glob.glob('*.shp')

In [22]:
# make sure final shapefile is skipped here if it already exits
length_file_name_final = len(roi_name + '_' + start_date.replace('-', '')) + 9 + 4 # add end date and file extension to file name length
shape_files = [i for i in shape_files if len(i) < length_file_name_final]

In [23]:
len(shape_files)

45

In [24]:
shape_files_gdf = gpd.GeoDataFrame(columns=['geometry', 'date'], geometry='geometry')

for shape_file in shape_files:
  shape_file_gdf_temp = gpd.read_file(shape_file)
  shape_file_gdf_temp['date'] = str(datetime.datetime.strptime(shape_file[17:25], "%Y%m%d").date())
  shape_file_gdf_temp = shape_file_gdf_temp.drop(['count', 'water'], axis=1)
  shape_files_gdf = pd.concat([shape_files_gdf, shape_file_gdf_temp])

In [25]:
shape_files_gdf = shape_files_gdf.reset_index(drop=True)

In [26]:
shape_files_gdf

,geometry,date
0,"POLYGON ((24.74948 -20.70922, 24.74948 -20.709...",2016-11-01
1,"POLYGON ((24.75245 -20.72422, 24.75245 -20.724...",2016-11-01
2,"MULTIPOLYGON (((24.75416 -20.72081, 24.75407 -...",2016-11-01
3,"POLYGON ((24.7546 -20.72665, 24.7546 -20.72656...",2016-11-01
4,"POLYGON ((24.75451 -20.7306, 24.75478 -20.7306...",2016-11-01
...,...,...
384718,"POLYGON ((25.17115 -20.70051, 25.17169 -20.700...",2017-04-23
384719,"POLYGON ((25.17573 -20.71973, 25.17591 -20.719...",2017-04-23
384720,"POLYGON ((25.1787 -20.73797, 25.17888 -20.7379...",2017-04-23
384721,"POLYGON ((25.17933 -20.7394, 25.17933 -20.7393...",2017-04-23


In [27]:
shape_files_gdf.groupby('date').size()

,0
date,
2016-11-01,22918
2016-11-06,6169
2016-11-13,14274
2016-11-18,7230
2016-11-25,14820
2016-11-30,11774
2016-12-07,29193
2016-12-12,9203
2016-12-19,27709


In [28]:
# identify invalid geometries
shape_files_gdf['is_valid'] = shape_files_gdf['geometry'].is_valid
print("Number of invalid geometries:", shape_files_gdf[~shape_files_gdf['is_valid']].shape[0])

Number of invalid geometries: 2


In [29]:
# repair invalid geometries
invalid_mask = ~shape_files_gdf['geometry'].is_valid
shape_files_gdf.loc[invalid_mask, 'geometry'] = shape_files_gdf.loc[invalid_mask, 'geometry'].buffer(0)

In [30]:
# recheck for validity
assert shape_files_gdf['geometry'].is_valid.all(), "Some geometries are still invalid!"

In [31]:
file_name_final = roi_name + '_' + str(min(shape_files_gdf['date'])).replace('-', '') + '_' + str(max(shape_files_gdf['date'])).replace('-', '') + '.shp'
shape_files_gdf.to_file(file_name_final)